In [1]:
import numpy as np

# This program will simulate particles moving in a circular path at a constant speed. 
# The purpose of this simulation is to improve performance of python program.

# This benchmark is to check how our code is performing
from random import uniform

In [2]:
def c_evolve(r_i, ang_speed_i, timestep, nsteps):
    v_i = np.empty_like(r_i)
    
    for i in range(nsteps):
        # 1. Calculate the direction
        norm_i = np.sqrt((r_i**2).sum(axis=1))
        v_i = r_i[:, [1, 0]]
        v_i[:, 0] *= -1
        v_i /= norm_i[:, np.newaxis]
        
        d_i = timestep * ang_speed_i[:, np.newaxis] * v_i
        
        r_i += d_i

In [6]:

class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel


# To simulate the particles
class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    # The first method utilizes numpy for simulation
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt/timestep)
        # nsteps = 1000 if dt = 0.01

        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
        
        for i in range(nsteps):
            
            norm_ii = (r_i ** 2).sum(axis=1)
            norm_i = np.sqrt(norm_ii)
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] *= -1
            v_i /= norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
            
            for i, p in enumerate(self.particles):
                p.x, p.y = r_i[i]

    # The second method            
    def evolve_cython(self, dt):
        timestep = 0.00001
        nsteps = int(dt/timestep)
        # nsteps = 1000 if dt = 0.01

        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_speed_i = np.array([p.ang_vel for p in self.particles])
        
        c_evolve(r_i, ang_speed_i, timestep, nsteps)
        
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]


In [7]:
def benchmark(npart=100, method='cython'):
    particles = [
        Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0))
        for i in range(npart)
    ]

    simulator = ParticleSimulator(particles)
    
    if method == 'cython':
        simulator.evolve_cython(0.1)
        
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

In [8]:

if __name__ == '__main__':
    benchmark(10, "cython")
    #pass

# To check performance use ipython: 
# %timeit benchmark(100, "python")
# and
# %timeit benchmark(100, "numpy")

# Result: You will see that numpy version is better for high number of particles.

In [9]:
%timeit benchmark(10, "cython")

230 ms ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit benchmark(10, 'numpy')

344 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
